In [32]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats, leaguedashteamclutch, leaguegamelog, boxscoreplayertrackv2, playercareerstats, teamplayerdashboard, leaguestandings
from nba_api.stats.static import players, teams
import time
pd.set_option('display.max_columns', None)
from IPython import display as ICD
import warnings
from datetime import date
import pickle
warnings.filterwarnings('ignore')

In [33]:
abbrv_mapping = {'DEN':"1610612743", 
           'SAS':"1610612759", 
           'IND':"1610612754", 
           'MEM':"1610612763", 
           'NOP':"1610612740", 
           'CHI':"1610612741", 
           'MIN':"1610612750", 
           'MIA':"1610612748", 
           'ATL':"1610612737",
           'ORL':"1610612753", 
           'SAC':"1610612758", 
           'UTA':"1610612762", 
           'CHA':"1610612766", 
           'WAS':"1610612764", 
           'HOU':"1610612745", 
           'PHX':"1610612756", 
           'OKC':"1610612760", 
           'POR':"1610612757",
           'LAC':"1610612746", 
           'GSW':"1610612744", 
           'PHI':"1610612755", 
           'LAL':"1610612747", 
           'BKN':"1610612751", 
           'TOR':"1610612761", 
           'BOS':"1610612738", 
           'MIL':"1610612749", 
           'DET':"1610612765",
           'CLE':"1610612739", 
           'NYK':"1610612752",
           'DAL':"1610612742"}

name_mapping = {'Nuggets':"1610612743", 
           'Spurs':"1610612759", 
           'Pacers':"1610612754", 
           'Memphis':"1610612763", 
           'Pelicans':"1610612740", 
           'Bulls':"1610612741", 
           'Timberwolves':"1610612750", 
           'Heat':"1610612748", 
           'Hawks':"1610612737",
           'Magic':"1610612753", 
           'Kings':"1610612758", 
           'Jazz':"1610612762", 
           'Hornets':"1610612766", 
           'Wizards':"1610612764", 
           'Rockets':"1610612745", 
           'Suns':"1610612756", 
           'Thunder':"1610612760", 
           'Trail Blazers':"1610612757",
           'Clippers':"1610612746", 
           'Warriors':"1610612744", 
           '76ers':"1610612755", 
           'Lakers':"1610612747", 
           'Nets':"1610612751", 
           'Raptors':"1610612761", 
           'Celtics':"1610612738", 
           'Bucks':"1610612749", 
           'Pistons':"1610612765",
           'Cavaliers':"1610612739", 
           'Knicks':"1610612752",
           'Mavericks':"1610612742"}

In [34]:
def clean_all_playoff_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season_type_all_star='Playoffs', season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

def clean_all_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

all_playoff_games = clean_all_playoff_games('2019-20')
all_regseason_playoff_games = clean_all_games('2019-20')
all_reg_season_playoff_games = pd.concat([all_playoff_games, all_regseason_playoff_games]).sort_values('GameDate',ascending = False)
all_reg_season_playoff_games.head(20)

,GameID,GameDate,TeamID,OpponentTeamID,MATCHUP,PTS,OPTS
8,0041900101,2020-08-18,1610612753,1610612749,ORL @ MIL,122,110
10,0041900131,2020-08-18,1610612754,1610612748,IND vs. MIA,101,113
11,0041900131,2020-08-18,1610612748,1610612754,MIA @ IND,113,101
12,0041900171,2020-08-18,1610612760,1610612745,OKC @ HOU,108,123
13,0041900171,2020-08-18,1610612745,1610612760,HOU vs. OKC,123,108
14,0041900141,2020-08-18,1610612757,1610612747,POR @ LAL,100,93
15,0041900141,2020-08-18,1610612747,1610612757,LAL vs. POR,93,100
9,0041900101,2020-08-18,1610612749,1610612753,MIL vs. ORL,110,122
4,0041900151,2020-08-17,1610612746,1610612742,LAC vs. DAL,118,110
6,0041900161,2020-08-17,1610612762,1610612743,UTA @ DEN,125,135


In [35]:
player_rotation = pd.read_csv('players_rotation.csv')
players_df = pd.read_csv('players.csv')
first_game_missing_mins = player_rotation.groupby('TEAM_ID').sum()['MIN'] / players_df.groupby('TEAM_ID').sum()['MIN']

In [36]:
def missing_mins(curr_game_id, team_id, season_name):
    '''
    missing_mins: gets the fraction of the season's minutes played by injured/inactive/traded players
    '''
    time.sleep(0.5)
    # get the season stats of the team
    season_stats = teamplayerdashboard.TeamPlayerDashboard(team_id, season=season_name).get_data_frames()[1]
    
    # get the game currently being assessed
    curr_game = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=curr_game_id).get_data_frames()[0]
    
    player_game_stats = curr_game[curr_game['TEAM_ID'] == int(team_id)]
    combined = pd.merge(season_stats, player_game_stats, how='left', on='PLAYER_ID')
    
    total_mins = combined['MIN_x'].sum()
    missing_mins = combined[(combined['MIN_y'] == "0:00") | (combined['MIN_y'].isna())]['MIN_x'].sum()
    return (total_mins - missing_mins) / total_mins

In [37]:
games = pd.read_csv('predictions/games.csv')
games['Date'] = games['Date'].astype('datetime64')
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual
0,2020-08-17,NaN,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,NaN,NaN
1,2020-08-17,NaN,1.610613e+09,1.610613e+09,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,NaN,NaN
2,2020-08-17,NaN,1.610613e+09,1.610613e+09,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,NaN,NaN
3,2020-08-17,NaN,1.610613e+09,1.610613e+09,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,NaN,NaN
4,2020-08-18,NaN,1.610613e+09,1.610613e+09,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,NaN,NaN
5,2020-08-18,NaN,1.610613e+09,1.610613e+09,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,NaN,NaN
6,2020-08-18,NaN,1.610613e+09,1.610613e+09,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,NaN,NaN
7,2020-08-18,NaN,1.610613e+09,1.610613e+09,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,NaN,NaN
8,NaT,NaN,NaN,NaN,Nuggets,Jazz,NaN,NaN,NaN,NaN,NaN,NaN
9,NaT,NaN,NaN,NaN,Raptors,Nets,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
def get_gameid(row):
    if(not pd.isna(row['GameID'])):
        return row['GameID']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = all_playoff_games[all_playoff_games['OpponentTeamID'] == row['OpponentTeamID']]
    temp = temp[temp['GameDate'].apply(lambda x: x.date() == row['Date'].date())]
    if(len(temp) != 0):
        return temp['GameID'].iloc[0]
    return row['GameID']
def get_game_info(row, attribute, alt_attribute):
    if(pd.isna(row['GameID'])):
        return row[alt_attribute]
    try:
        return all_playoff_games[(all_playoff_games['GameID'] == row['GameID']) & (all_playoff_games['TeamID'] == row['TeamID'])][attribute].iloc[0]
    except:
        return row[alt_attribute]
games['TeamID'] = games['TeamName'].apply(lambda x: int(name_mapping[x]))
games['OpponentTeamID'] = games['OpponentTeamName'].apply(lambda x: int(name_mapping[x]))
games['GameID'] = games.apply(lambda x: get_gameid(x), axis = 1)
games['Date'] = games.apply(lambda x: get_game_info(x, 'GameDate', 'Date'), axis = 1)
games['TeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'PTS', 'TeamPointsActual'), axis = 1)
games['OpponentTeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'OPTS', 'OpponentTeamPointsActual'), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual
0,2020-08-17,0041900161,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0
1,2020-08-17,0041900111,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0
2,2020-08-17,0041900121,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0
3,2020-08-17,0041900151,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0
4,2020-08-18,0041900101,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0
5,2020-08-18,0041900131,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0
6,2020-08-18,0041900171,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0
7,2020-08-18,0041900141,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0
8,NaT,NaN,1610612743,1610612762,Nuggets,Jazz,NaN,NaN,NaN,NaN,NaN,NaN
9,NaT,NaN,1610612761,1610612751,Raptors,Nets,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def get_series_lead(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series)
    games_won = len(series[series['PTS'] > series['OPTS']])
    return 2 * games_won - len_series

def get_series_win_percent(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series) + 1
    games_won = len(series[series['PTS'] > series['OPTS']])
    return games_won/len_series

In [40]:
# creating the games dataframe
games_copy = games[pd.isna(games['TeamWinProba'])][['Date','TeamID', 'OpponentTeamID']]
games_copy_reversed = games_copy.copy()
games_copy_reversed['TeamID'] = games_copy['OpponentTeamID']
games_copy_reversed['OpponentTeamID'] = games_copy['TeamID']

all_games = pd.concat([games_copy, games_copy_reversed]).sort_index().reset_index(drop=True)

reg_season_ratings = leaguestandings.LeagueStandings(season='2019-20', season_type = 'Regular Season').get_data_frames()[0]
reg_season_ratings = reg_season_ratings[['TeamID','PointsPG','OppPointsPG']]

#ratings
all_games = pd.merge(all_games, reg_season_ratings, how='left', on='TeamID')
all_games = pd.merge(all_games, reg_season_ratings, how='left', left_on='OpponentTeamID', right_on='TeamID', suffixes=['','Opponent'])

all_games['SeriesGP'] = all_games.apply(lambda x: 1 + len(all_playoff_games[(all_playoff_games['TeamID'] == x['OpponentTeamID']) & (all_playoff_games['OpponentTeamID'] == x['TeamID'])]), axis = 1) 
all_games['SeriesLead'] = all_games.apply(lambda x: get_series_lead(all_playoff_games, x), axis = 1) 
all_games['SeriesWinPercent'] = all_games.apply(lambda x: get_series_win_percent(all_playoff_games, x), axis = 1) 





all_games['PrevPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['PTS'].iloc[0], axis = 1) 
all_games['PrevOPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['OPTS'].iloc[0], axis = 1) 
all_games['PrevMargin'] = all_games['PrevPTS'] - all_games['PrevOPTS']


all_games['MissingMins'] = all_games.apply(lambda x: missing_mins(all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['GameID'].iloc[0], x['TeamID'], '2019-20'), axis = 1)
# first_game_missing_mins_copy = first_game_missing_mins.to_frame().reset_index()
# first_game_missing_mins_copy.columns = ['TeamID', 'MissingMins']
# all_games = pd.merge(all_games, first_game_missing_mins_copy, how='left', on='TeamID')


playoff_experience = pd.read_csv('playoff_experience.csv')
all_games = pd.merge(all_games, playoff_experience, how = 'left', on = 'TeamID')
all_games


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins
0,NaT,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,2,1,0.5,135,125,10,0.701686,391.0,6895.763333
1,NaT,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,2,-1,0.0,125,135,-10,0.719797,396.0,6328.350000
2,NaT,1610612761,1610612751,112.8,106.5,1610612751,111.8,112.3,2,1,0.5,134,110,24,0.915800,397.0,13204.620000
3,NaT,1610612751,1610612761,111.8,112.3,1610612761,112.8,106.5,2,-1,0.0,110,134,-24,0.567490,404.0,7254.545000
4,NaT,1610612738,1610612755,113.7,107.3,1610612755,110.7,108.4,2,1,0.5,109,101,8,0.878936,379.0,5772.186667
5,NaT,1610612755,1610612738,110.7,108.4,1610612738,113.7,107.3,2,-1,0.0,101,109,-8,0.734108,403.0,7951.311667
6,NaT,1610612746,1610612742,116.3,109.9,1610612742,117.0,112.1,2,1,0.5,118,110,8,0.781317,412.0,13322.215000
7,NaT,1610612742,1610612746,117.0,112.1,1610612746,116.3,109.9,2,-1,0.0,110,118,-8,0.761565,415.0,3609.745000
8,NaT,1610612749,1610612753,118.7,108.6,1610612753,107.3,108.3,2,-1,0.0,110,122,-12,0.867271,446.0,16320.056667
9,NaT,1610612753,1610612749,107.3,108.3,1610612749,118.7,108.6,2,1,0.5,122,110,12,0.688603,393.0,4178.705000


In [41]:
# bias boosted monte carlo simulation

In [42]:
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
def get_weight(x, col_value, range_val):
    return 1/(00.1+abs((x-col_value)/range_val))
def get_rand_sample(df, col_name, col_value, sample_col_name, N, range_val):
    weights = df[col_name].apply(lambda x: get_weight(x,col_value, range_val))
    to_return = random.choices(population=list(df[sample_col_name]), 
                               weights=weights,
                               k=N)
    return to_return

In [43]:
pts_model = pickle.load( open( "pts_model", "rb" ) )
opts_model = pickle.load( open( "opts_model", "rb" ) )
range_vals = pickle.load( open( "range_vals", "rb" ) )
playoff_data = pd.read_csv('all_playoff_games.csv')
offense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','PointsPG','OppPointsPGOpponent','PrevPTS']
defense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','OppPointsPG','PointsPGOpponent','PrevOPTS']

In [44]:
def get_predictions(playoff_games, cols, predictor, N, model = None):
    predictions = playoff_games[['Date', 'TeamID','OpponentTeamID']]
    col_sum = []
    for x in cols:
        col_sum.append(x + "Sum")
    for col in cols:
        predictions[col] = playoff_games.apply(lambda x: get_rand_sample(playoff_data,col,x[col],predictor,N,range_vals[col]), axis = 1)
    for col in range(len(cols)):
        predictions[col_sum[col]] = predictions[cols[col]].apply(lambda x: sum(x)/N)
    X = predictions[col_sum]
    
    coefs = pd.Series(model.coef_, index = col_sum).abs()
    # predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: (np.array(x.values.tolist()) * coefs.values[:, None]).sum(axis=0)/coefs.sum(), axis = 1)
    predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: model.predict(np.array(x.values.tolist()).transpose()), axis = 1)
    predictions[predictor + 'LinearRegression'] = model.predict(X)
    temp = predictions[['TeamID'] + col_sum + [predictor+'LinearRegression']]
    temp['Name'] = temp['TeamID'].apply(lambda x: list(name_mapping.keys())[list(pd.Series(list(name_mapping.values())).astype(int)).index(x)])
    ICD.display(temp)
    temp.to_csv('predictions_intermed/' + predictor+date.today().strftime('%d-%m-%Y') + '.csv', index = False)
    return predictions[[predictor + 'Combined', predictor + 'LinearRegression']]

In [45]:
pts_df = get_predictions(all_games, offense_cols, 'AdjustedPTS', 10000, pts_model)
opts_df = get_predictions(all_games, defense_cols, 'AdjustedOPTS', 10000, opts_model)
final = pd.concat([all_games, pts_df, opts_df], axis = 1)
final['WinProba'] = final.apply(lambda x: np.count_nonzero(x['AdjustedPTSCombined'] > x['AdjustedOPTSCombined'])/1000, axis = 1)
final['WinProbaScaled'] = final.apply(lambda x: x['WinProba']* 1/final[((final['SeriesGP'] == x['SeriesGP'])) & (((final['TeamID'] == x['TeamID']) & (final['OpponentTeamID'] == x['OpponentTeamID'])) | ((final['TeamID'] == x['OpponentTeamID']) & (final['OpponentTeamID'] == x['TeamID'])))]['WinProba'].sum(), axis = 1)
final['WinProba2x'] = final.apply(lambda x: max(0,min(1,2*x['WinProbaScaled']-0.5)), axis = 1)
ICD.display(final)
final = final[['Date','TeamID','OpponentTeamID','AdjustedPTSLinearRegression','AdjustedOPTSLinearRegression', 'WinProba2x']]

,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,PointsPGSum,OppPointsPGOpponentSum,PrevPTSSum,AdjustedPTSLinearRegression,Name
0,1610612743,105.035831,106.212361,107.238300,106.510139,106.426734,105.534373,106.275674,106.327233,105.822247,107.335939,107.770464,Nuggets
1,1610612762,105.078615,106.159346,107.182841,105.255026,105.757555,105.501167,105.584459,106.600239,105.975425,107.333290,107.143506,Jazz
2,1610612761,105.256902,106.100121,107.173469,106.485092,106.470619,106.833318,106.345275,106.881829,107.223090,107.329134,114.166629,Raptors
3,1610612751,105.195768,106.029870,107.302600,105.278377,106.118964,105.472491,105.752194,106.641918,106.056682,106.807307,106.746471,Nets
4,1610612738,105.256854,105.983861,107.416692,106.581727,106.300758,105.990629,106.075639,107.318588,106.080310,106.652092,110.313370,Celtics
5,1610612755,105.158163,106.180009,107.222978,105.291773,106.012781,105.585538,105.548995,106.233632,106.219237,105.463802,104.439060,76ers
6,1610612746,105.386521,105.979763,107.289167,106.264912,106.474358,106.220115,106.019544,108.562979,107.170106,107.563794,115.315608,Clippers
7,1610612742,105.610075,106.046999,107.268476,105.050255,105.836127,105.610909,105.555425,108.496055,106.620372,106.924162,111.309140,Mavericks
8,1610612749,105.860377,105.806136,107.054317,105.333577,106.028964,106.317162,105.893635,108.411102,106.083701,106.921762,112.237086,Bucks
9,1610612753,105.501607,105.989301,107.252057,106.437247,106.345305,105.082284,106.280215,104.747013,105.877070,107.481755,105.254838,Magic


,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,OppPointsPGSum,PointsPGOpponentSum,PrevOPTSSum,AdjustedOPTSLinearRegression,Name
0,1610612743,106.260950,106.027434,107.273733,105.276311,105.117966,106.143197,103.895732,106.094096,106.376341,107.528082,106.874231,Nuggets
1,1610612762,106.408231,106.320753,107.342588,106.521935,107.089106,106.092664,108.090874,105.942186,106.450360,107.529119,115.783769,Jazz
2,1610612761,105.448745,106.003632,107.517022,105.026127,104.965776,106.219173,102.364338,105.952882,106.376670,106.389552,100.522700,Raptors
3,1610612751,106.267674,105.837814,107.238424,106.512052,107.328245,105.637427,109.714502,107.262762,106.657324,107.737331,122.202364,Nets
4,1610612738,106.535066,106.008330,107.258099,105.174128,104.927966,106.097303,104.333918,106.013192,106.299966,105.551408,104.985801,Celtics
5,1610612755,106.498780,106.320328,107.391010,106.422270,107.217044,106.002751,107.817347,105.815667,107.373603,106.330622,113.894819,76ers
6,1610612746,105.784041,106.076233,107.212069,105.319199,104.944113,105.319500,104.352072,106.087718,108.614929,106.726976,107.898112,Clippers
7,1610612742,106.866808,106.187712,107.257857,106.614204,107.228730,105.548713,107.773858,107.163744,108.301327,107.566086,118.565717,Mavericks
8,1610612749,105.360734,105.899924,107.360793,106.455225,107.013372,106.167013,108.421041,105.875535,104.345165,107.466758,113.188670,Bucks
9,1610612753,106.822105,105.991664,107.239743,105.160322,105.012080,106.272439,103.562728,105.918872,108.672653,106.628195,107.637052,Magic


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins,AdjustedPTSCombined,AdjustedPTSLinearRegression,AdjustedOPTSCombined,AdjustedOPTSLinearRegression,WinProba,WinProbaScaled,WinProba2x
0,NaT,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,2,1,0.5,135,125,10,0.701686,391.0,6895.763333,"[111.44379310917316, 65.75130822280721, 22.411...",107.770464,"[19.90599564952322, 74.6814835562252, 208.2423...",106.874231,5.068,0.527312,0.554625
1,NaT,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,2,-1,0.0,125,135,-10,0.719797,396.0,6328.350000,"[155.26776036452293, 45.7021823391251, 153.926...",107.143506,"[236.12809877609084, 109.60433104830486, 65.07...",115.783769,4.543,0.472688,0.445375
2,NaT,1610612761,1610612751,112.8,106.5,1610612751,111.8,112.3,2,1,0.5,134,110,24,0.915800,397.0,13204.620000,"[81.79761813109144, 104.37389063850856, 139.71...",114.166629,"[113.12062947422373, 87.25346598547776, 107.02...",100.522700,5.678,0.574289,0.648579
3,NaT,1610612751,1610612761,111.8,112.3,1610612761,112.8,106.5,2,-1,0.0,110,134,-24,0.567490,404.0,7254.545000,"[76.11178504738655, 169.8137435379624, 118.497...",106.746471,"[96.89764635895472, 101.12432519197318, 143.77...",122.202364,4.209,0.425711,0.351421
4,NaT,1610612738,1610612755,113.7,107.3,1610612755,110.7,108.4,2,1,0.5,109,101,8,0.878936,379.0,5772.186667,"[80.16288701100893, 97.84568373002139, 193.991...",110.313370,"[95.47265738654096, 54.25624154131174, 158.082...",104.985801,5.225,0.537165,0.574329
5,NaT,1610612755,1610612738,110.7,108.4,1610612738,113.7,107.3,2,-1,0.0,101,109,-8,0.734108,403.0,7951.311667,"[79.6457220422983, 56.15686166622254, 118.6245...",104.439060,"[117.6416934682502, 234.51495121832704, -28.63...",113.894819,4.502,0.462835,0.425671
6,NaT,1610612746,1610612742,116.3,109.9,1610612742,117.0,112.1,2,1,0.5,118,110,8,0.781317,412.0,13322.215000,"[92.67279434134616, 150.23040717652407, 157.73...",115.315608,"[194.12205764439, 96.59645841487463, 16.289194...",107.898112,5.374,0.536060,0.572120
7,NaT,1610612742,1610612746,117.0,112.1,1610612746,116.3,109.9,2,-1,0.0,110,118,-8,0.761565,415.0,3609.745000,"[179.9907858218612, 12.692632590177709, 3.7115...",111.309140,"[48.68380706775224, 134.48630490723986, 92.241...",118.565717,4.651,0.463940,0.427880
8,NaT,1610612749,1610612753,118.7,108.6,1610612753,107.3,108.3,2,-1,0.0,110,122,-12,0.867271,446.0,16320.056667,"[54.15844265951341, 59.41883623499439, 113.908...",112.237086,"[90.3012828404801, 223.4856196548592, 15.77451...",113.188670,4.963,0.501972,0.503945
9,NaT,1610612753,1610612749,107.3,108.3,1610612749,118.7,108.6,2,1,0.5,122,110,12,0.688603,393.0,4178.705000,"[196.94143779855585, 82.44463885372079, 111.47...",105.254838,"[140.66766723583464, -10.375372585260834, 80.2...",107.637052,4.924,0.498028,0.496055


In [46]:
for x in range(1, len(final),2):
    temp = final['AdjustedPTSLinearRegression'].iloc[x]
    final['AdjustedPTSLinearRegression'].iloc[x] = final['AdjustedOPTSLinearRegression'].iloc[x]
    final['AdjustedOPTSLinearRegression'].iloc[x] = temp
final =final.groupby(np.arange(len(final))//2).agg({'Date': 'first',
                                            'TeamID':'first',
                                             'OpponentTeamID':'first',
                                             'AdjustedPTSLinearRegression':'mean',
                                             'AdjustedOPTSLinearRegression':'mean',
                                             'WinProba2x':'first'})

In [47]:
import scipy.stats
norm_dist = scipy.stats.norm(0, 10)
final['ProbaNorm'] = final.apply(lambda x: norm_dist.cdf(x['AdjustedPTSLinearRegression'] - x['AdjustedOPTSLinearRegression']), axis = 1)
final

,Date,TeamID,OpponentTeamID,AdjustedPTSLinearRegression,AdjustedOPTSLinearRegression,WinProba2x,ProbaNorm
0,NaT,1610612743,1610612762,111.777116,107.008869,0.554625,0.683257
1,NaT,1610612761,1610612751,118.184497,103.634586,0.648579,0.927164
2,NaT,1610612738,1610612755,112.104094,104.712430,0.574329,0.770097
3,NaT,1610612746,1610612742,116.940663,109.603626,0.572120,0.768435
4,NaT,1610612749,1610612753,109.937069,109.221754,0.503945,0.528513
5,NaT,1610612754,1610612748,102.270050,112.125324,0.395589,0.162183
6,NaT,1610612745,1610612760,117.005689,108.005574,0.585371,0.815943
7,NaT,1610612747,1610612757,108.130878,109.352512,0.493388,0.451385


In [48]:
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual
0,2020-08-17,0041900161,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0
1,2020-08-17,0041900111,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0
2,2020-08-17,0041900121,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0
3,2020-08-17,0041900151,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0
4,2020-08-18,0041900101,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0
5,2020-08-18,0041900131,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0
6,2020-08-18,0041900171,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0
7,2020-08-18,0041900141,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0
8,NaT,NaN,1610612743,1610612762,Nuggets,Jazz,NaN,NaN,NaN,NaN,NaN,NaN
9,NaT,NaN,1610612761,1610612751,Raptors,Nets,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
def get_game_info_calculated(row, final_col, games_col):
    if(not pd.isna(row[games_col])):
        return row[games_col]
    temp = final[final['TeamID'] == row['TeamID']]
    temp = final[final['OpponentTeamID'] == row['OpponentTeamID']]
    return temp[final_col].iloc[0]

games['TeamWinProba'] = games.apply(lambda x: get_game_info_calculated(x, 'ProbaNorm', 'TeamWinProba'), axis = 1)
games['OpponentTeamWinProba'] = games.apply(lambda x: 1 - x['TeamWinProba'], axis = 1)
games['TeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedPTSLinearRegression', 'TeamPoints'), axis = 1)
games['OpponentTeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedOPTSLinearRegression', 'OpponentTeamPoints'), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual
0,2020-08-17,0041900161,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0
1,2020-08-17,0041900111,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0
2,2020-08-17,0041900121,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0
3,2020-08-17,0041900151,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0
4,2020-08-18,0041900101,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0
5,2020-08-18,0041900131,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0
6,2020-08-18,0041900171,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0
7,2020-08-18,0041900141,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0
8,NaT,NaN,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777116,107.008869,NaN,NaN
9,NaT,NaN,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,NaN,NaN


In [51]:
today = date.today()

pd.read_csv('predictions/games.csv').to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_before.csv', index = False)
games.to_csv('predictions/games.csv', index = False)
games.to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_after.csv', index = False)